In [2]:
import newspaper
from newspaper import Article, Config
from IPython.core.debugger import set_trace
import requests
import json
import pandas as pd
# from dateutil.parser import *
from tqdm.autonotebook import tqdm
tqdm.pandas()

import os
import sys
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

# import django
# django.setup()

# from data.models import Newsdata

In [9]:
# 필요없는 신문사 지우기: nbc - text가 잘 extract 안됨 
Newsdata.objects.filter(publisher='nbc').delete();

In [20]:
# url 끝이 / 로 안끝나게 하기
for obj in tqdm(Newsdata.objects.filter(url__endswith='/')):
    obj.url = obj.url[:-1]
    obj.save()

In [13]:
# url이 항상 https://로 시작하게(www없이)
# 그리고 ?mod= 는 제거
# for obj in tqdm(Newsdata.objects.all()):
for obj in tqdm(Newsdata.objects.filter(url__contains='?')):
    url = obj.url.replace('http://', 'https://')
    url = url.replace('https://www', 'https://')
    
    try: url = url[:url.index('?mod=')]
    except: pass
    
    try: url = url[:url.index('?il=0')]
    except: pass
    
    try: url = url[:url.index('?iid=')]
    except: pass
        
    obj.url = url
    obj.publish_date = None
    obj.save()

In [16]:
for obj in Newsdata.objects.filter(url__contains='?source='):
    print(obj.url)

https://fool.com/investing/2019/08/16/sorry-wework-isnt-at-all-like-iwg-or-regus.aspx?source=iedfolrf0000001
https://fool.com/investing/2019/08/16/sorry-wework-isnt-at-all-like-iwg.aspx?source=iedfolrf0000001
https://fool.com/investing/2019/08/16/starbucks-vs-luckin-what-to-consider.aspx?source=iedfolrf0000001


In [17]:
ar = Article('https://fool.com/investing/2019/08/16/sorry-wework-isnt-at-all-like-iwg-or-regus.aspx')

In [18]:
ar.url

'https://fool.com/investing/2019/08/16/sorry-wework-isnt-at-all-like-iwg-or-regus.aspx'

In [71]:
# 동일 url 모두 제거
for url in tqdm(set(Newsdata.objects.values_list('url', flat=True))):
    objs = Newsdata.objects.filter(url=url)
    if len(objs)>1:
        [obj.delete() for obj in objs[1:]]
        print(url)

https://reuters.com/article/us-japan-economy-trade/japans-exports-slip-for-eighth-month-sales-to-china-drop-as-recession-fears-grow-idUSKCN1V9015
https://reuters.com/article/us-usa-economy/trump-not-ready-for-china-trade-deal-dismisses-recession-fears-idUSKCN1V80HS
https://washingtontimes.com/news/2019/aug/18/thailand-animal-trafficking-cases-difficult-prosec
https://cbsnews.com/news/hong-kong-protests-1-7-million-protesters-march-victoria-park-latest-updates-today-2019-08-18
https://morningstar.com/articles/938581/9-exceptional-and-plucky-core-bond-funds
https://reuters.com/article/us-hongkong-protests/more-mass-protests-planned-in-hong-kong-after-peaceful-weekend-of-rallies-idUSKCN1V900Z
https://washingtontimes.com/news/2019/aug/18/robert-davis-making-strong-case-roster-spot
https://washingtontimes.com/news/2019/aug/16/being-human
https://morningstar.com/articles/799730/investing-tips-for-twenty-and-thirtysomethings
https://morningstar.com/articles/940316/an-etf-for-exposure-to-short

https://reuters.com/article/us-argentina-emerging-markets-analysis/from-tsunami-to-ripples-other-emerging-markets-surf-argentinas-latest-shock-idUSKCN1V61HI
https://reuters.com/article/us-japan-economy-trade/japans-exports-slip-for-eighth-month-sales-to-china-drop-as-recession-fears-grow-idUSKCN1V9015?il=0
https://washingtontimes.com/news/2019/aug/18/north-korea-china-generals-meet-beijing
https://morningstar.com/articles/801233/an-investing-road-map-if-youre-in-your-40s-and-50s
https://reuters.com/article/us-usa-election-trump/trump-affirms-that-mike-pence-will-be-2020-running-mate-idUSKCN1V80NU
https://morningstar.com/articles/934609/building-a-low-cost-fossil-fuel-free-etf-portfolio
https://washingtontimes.com/news/2019/aug/18/owen-jones-guardian-journalist-says-he-was-victim-
https://nytimes.com/2019/08/19/arts/queer-eye-kondo-makeover.html
https://washingtontimes.com/news/2019/aug/18/nancy-pelosi-calls-donald-trump-foreign-aid-cuts-i
https://washingtontimes.com/news/2019/aug/18/pr

In [78]:
# publisher 이름 재설정 
for obj in tqdm(Newsdata.objects.all()):
    publisher = obj.publisher
    if 'huffing' in publisher:
        obj.publisher = 'huffingtonpost'
    elif 'investing' in publisher:
        obj.publisher = 'investingcom'
    elif 'fox'==publisher:
        obj.publisher = 'foxnews'
    elif 'fox_business'==publisher:
        obj.publisher = 'foxbusiness'
    elif 'business_insider'==publisher:
        obj.publisher = 'businessinsider'
    elif 'newyork_times'==publisher:
        obj.publisher = 'nyt'
    elif 'washington_times'==publisher:
        obj.publisher = 'washingtontimes'
    elif 'washington_post'==publisher:
        obj.publisher = 'washingtonpost'
    elif 'market_watch'==publisher:
        obj.publisher = 'marketwatch'
        
    obj.save()

In [12]:
newsdata = Newsdata.objects.values('title','text','url','publisher')

In [18]:
with open('newsdata.json', 'w') as f:
    json.dump(list(newsdata), f)

In [7]:
urls = Newsdata.objects.values_list('url', flat=True)
with open('urls.json', 'w') as f:
    json.dump(list(urls), f)

In [25]:
src = newspaper.build('https://cnn.com', config=config)

In [29]:
src.articles[110].url

'https://cnn.com/2019/08/26/tech/huawei-futurewei-us-future/index.html'

In [5]:
config = Config()
config.fetch_images = False
config.memoize_articles = False
config.language = 'en'

In [4]:
newspaper.popular_urls()

['http://www.huffingtonpost.com',
 'http://cnn.com',
 'http://www.time.com',
 'http://www.ted.com',
 'http://pandodaily.com',
 'http://www.cnbc.com',
 'http://www.mlb.com',
 'http://www.pcmag.com',
 'http://www.foxnews.com',
 'http://theatlantic.com',
 'http://www.bbc.co.uk',
 'http://www.vice.com',
 'http://www.elle.com',
 'http://www.vh1.com',
 'http://espnf1.com',
 'http://espn.com',
 'http://www.npr.org',
 'http://www.sfgate.com',
 'http://www.glamour.com',
 'http://www.whosdatedwho.com',
 'http://kotaku.com',
 'http://thebostonchannel.com',
 'http://www.suntimes.com',
 'http://www.businessinsider.com',
 'http://www.rivals.com',
 'http://thebusinessjournal.com',
 'http://www.newrepublic.com',
 'http://allthingsd.com',
 'http://www.topgear.com',
 'http://thecitizen.com',
 'http://www.ign.com',
 'http://www.sci-news.com',
 'http://www.morningstar.com',
 'http://www.variety.com',
 'http://thebottomline.as.ucsb.edu',
 'http://www.gamefaqs.com',
 'http://blog.searchenginewatch.com',
 'h

In [62]:
pubs = {
    'huffingtonpost':  'https://huffpost.com', #
    'cnn':             'https://cnn.com', #
    'investingcom':    'https://investing.com', #
    'politico':        'https://politico.com', #
    'time':            'https://time.com', # 다시
    'cnbc':            'https://cnbc.com', #
    'foxnews':         'https://foxnews.com', #
    'foxbusiness':     'https://foxbusiness.com', # 다시
    'bbc':             'https://bbc.com', # 다시
    'businessinsider': 'https://businessinsider.com', #
    'morningstar':     'https://morningstar.com', #
    'wsj':             'https://wsj.com', # ?mod=rsswn
    'nyt':             'https://nytimes.com', #
    'guardian':        'https://theguardian.com', #
    'reuters':         'https://reuters.com', #
    'washingtontimes': 'https://washingtontimes.com', #
    'washingtonpost':  'https://washingtonpost.com', #
    'cbs':             'https://cbsnews.com', #
    'marketwatch':     'https://marketwatch.com', #
    'atlantic':        'https://theatlantic.com', 
    'vice':            'https://vice.com', 
    'npr':             'https://npr.org', 
    'newrepublic':     'https://newrepublic.com', 
    'yahoo':           'https://yahoo.com', 
    'independent':     'https://independent.co.uk', 
    'heritage':        'https://heritage.org', 
    'zdnet':           'https://zdnet.com', 
    'townhall':        'https://townhall.com', 
    'abcnews':         'https://abcnews.go.com', 
    'hotair':          'https://hotair.com', 
    'cbc':             'https://cbc.ca', 
    'nymag':           'https://nymag.com', 
    'thestreet':       'https://thestreet.com', 
    'thinkprogress':   'https://thinkprogress.org', 
    'dailybeast':      'https://thedailybeast.com', 
    'realclearpolitcs':'https://realclearpolitics.com', 
    
    #'forbes':          'https://forbes.com', 
    #'hbr':             'https://hbr.org', 
    #'ft':              'https://ft.com', 
    #'economist':       'https://economist.com', 
}

In [9]:
def clean_url(publisher, url):
    url = url.replace('http://', 'https://')
    url = url.replace('https://www.', 'https://')
    
    if url[-1]=='/':
        url = url[:-1]
    
    if publisher=='wsj':
        try: url = url[:url.index('?mod=')]
        except: pass
        
        url += '?mod=rsswn'
        
    return url

In [11]:
s = time.time()
urls = set()
for publisher, domain in tqdm(pubs.items()):
    articles = newspaper.build(domain, config=config).articles
    urls |= {clean_url(publisher, article.url) for article in articles}

print(time.time()-s)


100.9901807308197


In [12]:
urls

{'https://cnn.com/style/article/hong-kong-protest-tattoo-intl-hnk-trnd/index.html',
 'https://cnn.com/style/article/russia-avant-garde-film-posters/index.html',
 'https://cnn.com/2019/07/26/sport/kargo-autonomous-ai-delivery-vehicle-supercharged-spt-intl/index.html',
 'https://id.investing.com/news/latest-news',
 'https://cnn.com/2019/08/26/investing/stock-market-insider-selling/index.html',
 'https://investing.com/news/economy-news/jerome-powell-just-locked-in-a-july-rate-cut-1919889#comments',
 'https://nl.investing.com/news/economy/chinas-mofcom-zegt-dat-beide-partijen-voorwaarden-scheppen-voor-onderhandelingen-130919',
 'https://huffpost.com/entry/martha-stewart-masters-the-fine-craft-of-throwing-shade-at-gwyneth-paltrow_n_5d66a81ce4b063c341f95b71',
 'https://cnn.com/2015/01/06/world/iyw-syria-resource-list/index.html',
 'https://pl.investing.com/analysis/mocne-wzrosty-na-kursie-ropy-przez-duzy-spadek-zapasow-surowca-200226574',
 'https://pt.investing.com/news/stock-market-news/tax

In [32]:
src = [newspaper.build(url, config=config).articles for publisher, domain in tqdm(pubs.items())]

In [43]:
ar = src['cnn'][100]
ar.download()
ar.parse()

In [52]:
src['cnn']

AttributeError: 'list' object has no attribute 'url'

In [19]:
# bulk_newsdata = []
i = 1

for publisher, articles in tqdm(list(src.items())[:]):
    for article in tqdm(articles):
        try:
            article.download()
            article.parse()

            publish_date = article.publish_date
            text = article.text

            if (text!='') and (publish_date is not None):
                obj = Newsdata()
                obj.id = i
                obj.text = article.text
                obj.authors = ','.join(article.authors)
                obj.url = article.url
                obj.title = article.title
                obj.publish_date = pd.Timestamp(article.publish_date)
                obj.publisher = publisher

                doc = nlp(article.text)
                obj.words = ','.join([str(noun) for noun in doc.noun_chunks])
                obj.save()
                
                i += 1

                #bulk_newsdata.append(obj)
                
        except:
            continue

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-08-18 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-08-17 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-08-16 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-08-13 00:00:00) while tim

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2018-10-17 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2018-09-28 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2018-09-07 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2017-09-14 00:00:00) while tim

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-07-11 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-07-09 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-07-06 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-07-05 00:00:00) while tim

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2018-06-04 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2018-05-01 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2018-04-30 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2017-11-28 00:00:00) while tim

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2014-05-29 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2015-08-17 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2015-06-23 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2017-12-11 00:00:00) while tim

...

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2016-01-12 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2016-01-06 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2016-01-05 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2016-01-04 00:00:00) while tim

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2015-05-28 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2015-05-26 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2015-05-21 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2015-05-20 00:00:00) while tim

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2011-11-18 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2014-05-06 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2014-05-08 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2012-01-16 00:00:00) while tim

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-04-24 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-03-12 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-07-04 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-04-07 00:00:00) while tim

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2018-10-10 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2016-06-14 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2018-11-12 00:00:00) while time zone support is active.
  RuntimeWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\django\db\models\fields\__init__.py:1421: RuntimeWarning: DateTimeField Newsdata.publish_date received a naive datetime (2019-04-29 00:00:00) while tim

In [242]:
#Newsdata.objects.bulk_create(bulk_newsdata);

In [1]:
import requests

In [71]:
headers = {"Authorization":"sBBqsGXiYgF0Db5OV5tAw-akap7JBapGCe67Tuuz0lhquXd0k1oQC4B5B91GiHcVn2pHZrSf1gT2PUujH1YaQA"}
endpoint = 'https://api.newsriver.io/v2/search?'
# qry = 'query=text:("Google tells workers to cool it on politics at work")&sortOrder=DESC&limit=15'
qry = 'query=url:"https://cbsnews.com/pictures"'
# qry = 'query=language:en AND website.domainName:"wsj.com"&sortOrder=DESC&limit=15'
url = endpoint + qry
response = requests.get(url, headers=headers)
out = response.json();out

[]

In [ ]:
title = "'Watershed moment': Israel critics seize on Trump-Netanyahu bromance after travel ban"
url = 'https://newsapi.org/v2/everything?qInTitle="{title}"&apiKey=bfb917bca91c435d9c7efd247d95227b&pageSize=100'.format(title=title)
resp = requests.get(url)
resp.json()

In [69]:
import asyncio
from functools import partial
import time

s = time.time()
n = len(pubs)
done = [0]
results = []

async def geturls(publisher, domain):
    newspaper_config = partial(newspaper.build, config=config)
    src = await loop.run_in_executor(None, newspaper_config, domain)
    articles = src.articles
    urls = {clean_url(publisher, article.url) for article in articles}
    
    done[0] += 1
    pct = '%.2f' % (done[0]/n*100)
    print('\r{publisher}... {pct}% completed          '.format(publisher=publisher, pct=pct), end='')
    return urls


async def main():
    fts = [asyncio.ensure_future(geturls(publisher, domain)) for publisher, domain in pubs.items()]
    return await asyncio.gather(*fts)


asyncio.set_event_loop(asyncio.new_event_loop())
loop = asyncio.get_event_loop()

try:
    # 다음 코드를 주피터에서 돌리려면, tornado를 downgrade 해야함
    # pip install tornado==4.5.3
    result = loop.run_until_complete(main())
    result = set.union(*result)

except Exception as ex:
    print(ex)

finally:
    loop.close()

e = time.time()
print('\n{0:.2f}초'.format(e - s))

investingcom... 100.00% completed             
91.48초


In [67]:
result;

In [68]:
len(result)

10263

In [78]:
import os.path
import glob

In [79]:
file = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), '**', 'newscloud.html')
file

'C:\\Users\\Sejin\\Documents\\GitHub\\protobed\\**\\newscloud.html'

In [80]:
pattern = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), '**', 'newscloud.html')

In [82]:
glob.glob(pattern, recursive=True)

['C:\\Users\\Sejin\\Documents\\GitHub\\protobed\\data\\templates\\data\\newscloud.html']

In [3]:
import articleDateExtractor

In [17]:
articleDateExtractor.extractArticlePublishedDate('https://www.politico.com/story/2019/08/29/republicans-trump-economy-anxiety-1476780')

Extracting date from https://www.politico.com/story/2019/08/29/republicans-trump-economy-anxiety-1476780


datetime.datetime(2028, 8, 19, 0, 0)

In [18]:
ar = Article('https://www.politico.com/story/2019/08/29/republicans-trump-economy-anxiety-1476780')
ar.download()
ar.parse()

In [20]:
ar.title

'Republicans grow anxious about the Trump economy'

In [21]:
title = "Republicans grow anxious about the Trump economy"
url = 'https://newsapi.org/v2/everything?qInTitle="{title}"&apiKey=bfb917bca91c435d9c7efd247d95227b&pageSize=100'.format(title=title)
resp = requests.get(url)
resp.json()

{'status': 'ok',
 'totalResults': 4,
 'articles': [{'source': {'id': 'politico', 'name': 'Politico'},
   'author': 'Burgess Everett',
   'title': 'Republicans grow anxious about the Trump economy',
   'description': "Trump's trade war with China could undermine GOP chances of holding the White House and Senate in 2020.",
   'url': 'https://www.politico.com/story/2019/08/29/republicans-trump-economy-anxiety-1476780',
   'urlToImage': 'https://static.politico.com/16/30/e63e94e8488a90d8218dc7bb34c8/190828-toomey-ap-773.jpg',
   'publishedAt': '2019-08-29T09:09:31Z',
   'content': 'Republican Sen. Patrick Toomey fears that trade uncertainty is contributing to an economic slowdown. | Jacquelyn Martin/AP Photo\r\nRepublicans have sat patiently with President Donald Trump on his tariff roller-coaster ride with China. Now theyre starting to f… [+7865 chars]'},
  {'source': {'id': None, 'name': 'Yahoo.com'},
   'author': 'By Burgess Everett',
   'title': 'Republicans grow anxious about the Trum

In [26]:
headers = {"Authorization":"sBBqsGXiYgF0Db5OV5tAw-akap7JBapGCe67Tuuz0lhquXd0k1oQC4B5B91GiHcVn2pHZrSf1gT2PUujH1YaQA"}
endpoint = 'https://api.newsriver.io/v2/search?'
qry = 'query=title:"Republicans grow anxious about the Trump economy"&sortOrder=DESC&limit=15'
# qry = 'query=language:en AND website.domainName:"wsj.com"&sortOrder=DESC&limit=15'
url = endpoint + qry
response = requests.get(url, headers=headers)
out = response.json();out

[{'id': 'plePuRZae336w7aAe8Vx7rWkiAf5VYVAD-S0zKDqnsDnZ5LXmnnTsF1QcrtXlj6L6rUDtlYwddpj8rtuO1EtEQ',
  'publishDate': '2019-08-29T00:00:00',
  'discoverDate': '2019-08-29T09:32:48.176+0000',
  'title': 'Republicans grow anxious about the Trump economy',
  'language': 'en',
  'text': "Republicans have sat patiently with President Donald Trump on his tariff roller-coaster ride with China. Now they’re starting to feel queasy. Trump argues his escalating trade war will force China to the table for a deal. But his ever-rising tariffs — and his market-rattling tweets — are increasingly alarming the GOP. Story Continued Below “There’s no question that trade uncertainty is contributing to the slowdown,” said Sen. Pat Toomey (R-Pa.), a leading free-trader. “We’re in a very good place. The danger is: Where are we going to be a year from now if concerns about trade continue to be an irritant to growth?” Particularly as the global economy cools, key Republicans say new levies on almost all Chinese go

In [24]:
len(out)

1